### `AutoGen` vs `Agent Framework` : Power of function calling with multiple interconnected tools! 
This code demonstrates how both frameworks enable agents to use external functions/tools to answer questions - a critical capability for building practical AI agents that can interact with APIs, databases, and services.

In [1]:
# Install required packages
! pip install autogen-agentchat autogen-ext[openai] agent-framework openai python-dotenv -q
print("✅ All packages installed successfully!")

✅ All packages installed successfully!


In [ ]:
# Import all required libraries
import asyncio
import os
import json
from typing import List, Dict, Optional, Annotated
from datetime import datetime, timedelta
from dataclasses import dataclass
from pydantic import BaseModel, Field
from dotenv import load_dotenv

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [3]:
load_dotenv()

True

In [4]:
# Configure API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

print("✅ API Key configured!")

✅ API Key configured!


In [5]:
# Define data models for travel booking

from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import datetime

class Flight(BaseModel):
    """Flight information."""
    flight_number: str
    airline: str
    departure: str
    arrival: str
    departure_time: str
    arrival_time: str
    price: float
    duration: str
    stops: int

class Hotel(BaseModel):
    """Hotel information."""
    name: str
    location: str
    rating: float
    price_per_night: float
    amenities: List[str]
    available_rooms: int

class TripItinerary(BaseModel):
    """Complete trip itinerary."""
    destination: str
    departure_date: str
    return_date: str
    flight_out: Optional[Flight] = None
    flight_return: Optional[Flight] = None
    hotel: Optional[Hotel] = None
    total_cost: float = 0.0
    booking_confirmed: bool = False

# Mock databases
FLIGHTS_DB = {
    "NYC-LAX": [
        Flight(
            flight_number="AA123",
            airline="American Airlines",
            departure="New York (JFK)",
            arrival="Los Angeles (LAX)",
            departure_time="08:00 AM",
            arrival_time="11:30 AM",
            price=320.00,
            duration="5h 30m",
            stops=0
        ),
        Flight(
            flight_number="UA456",
            airline="United Airlines",
            departure="New York (JFK)",
            arrival="Los Angeles (LAX)",
            departure_time="02:00 PM",
            arrival_time="05:45 PM",
            price=280.00,
            duration="5h 45m",
            stops=1
        )
    ],
    "NYC-MIA": [
        Flight(
            flight_number="DL789",
            airline="Delta Airlines",
            departure="New York (JFK)",
            arrival="Miami (MIA)",
            departure_time="10:00 AM",
            arrival_time="01:30 PM",
            price=220.00,
            duration="3h 30m",
            stops=0
        )
    ]
}

HOTELS_DB = {
    "Los Angeles": [
        Hotel(
            name="Hollywood Luxury Hotel",
            location="Hollywood, Los Angeles",
            rating=4.5,
            price_per_night=250.00,
            amenities=["Pool", "Gym", "Free WiFi", "Restaurant"],
            available_rooms=10
        ),
        Hotel(
            name="Downtown Budget Inn",
            location="Downtown, Los Angeles",
            rating=3.8,
            price_per_night=120.00,
            amenities=["Free WiFi", "Parking"],
            available_rooms=5
        )
    ],
    "Miami": [
        Hotel(
            name="Miami Beach Resort",
            location="South Beach, Miami",
            rating=4.8,
            price_per_night=350.00,
            amenities=["Beach Access", "Pool", "Spa", "Restaurant"],
            available_rooms=8
        )
    ]
}

BOOKINGS_DB = {}

print("✅ Data models and mock databases created!")
print(f"   📊 {len(FLIGHTS_DB)} flight routes available")
print(f"   🏨 {len(HOTELS_DB)} hotel destinations available")


✅ Data models and mock databases created!
   📊 2 flight routes available
   🏨 2 hotel destinations available


In [6]:
# Define comprehensive travel booking functions

def search_flights(
    origin: str,
    destination: str,
    date: str,
    passengers: int = 1
) -> str:
    """
    Search for available flights between two cities.
    
    Args:
        origin: Departure city (e.g., "New York", "NYC")
        destination: Arrival city (e.g., "Los Angeles", "LAX")
        date: Travel date in YYYY-MM-DD format
        passengers: Number of passengers (default: 1)
    
    Returns:
        JSON string with available flights
    """
    print(f"✈️  Searching flights: {origin} → {destination} on {date}")
    
    # Normalize city names
    route_key = f"{origin.upper()[:3]}-{destination.upper()[:3]}"
    
    flights = FLIGHTS_DB.get(route_key, [])
    
    if not flights:
        return json.dumps({
            "status": "no_flights",
            "message": f"No flights found for {origin} to {destination}"
        })
    
    # Convert to dict for JSON serialization
    flights_data = [
        {
            "flight_number": f.flight_number,
            "airline": f.airline,
            "departure": f.departure,
            "arrival": f.arrival,
            "departure_time": f.departure_time,
            "arrival_time": f.arrival_time,
            "price": f.price * passengers,
            "duration": f.duration,
            "stops": f.stops
        }
        for f in flights
    ]
    
    return json.dumps({
        "status": "success",
        "origin": origin,
        "destination": destination,
        "date": date,
        "passengers": passengers,
        "flights_found": len(flights_data),
        "flights": flights_data
    }, indent=2)


def search_hotels(
    city: str,
    check_in: str,
    check_out: str,
    guests: int = 2
) -> str:
    """
    Search for available hotels in a city.
    
    Args:
        city: City name (e.g., "Los Angeles", "Miami")
        check_in: Check-in date in YYYY-MM-DD format
        check_out: Check-out date in YYYY-MM-DD format
        guests: Number of guests (default: 2)
    
    Returns:
        JSON string with available hotels
    """
    print(f"🏨 Searching hotels in {city} ({check_in} to {check_out})")
    
    # Find hotels in the city
    hotels = None
    for city_name in HOTELS_DB:
        if city.lower() in city_name.lower() or city_name.lower() in city.lower():
            hotels = HOTELS_DB[city_name]
            break
    
    if not hotels:
        return json.dumps({
            "status": "no_hotels",
            "message": f"No hotels found in {city}"
        })
    
    # Calculate number of nights
    check_in_date = datetime.strptime(check_in, "%Y-%m-%d")
    check_out_date = datetime.strptime(check_out, "%Y-%m-%d")
    nights = (check_out_date - check_in_date).days
    
    # Convert to dict
    hotels_data = [
        {
            "name": h.name,
            "location": h.location,
            "rating": h.rating,
            "price_per_night": h.price_per_night,
            "total_price": h.price_per_night * nights,
            "nights": nights,
            "amenities": h.amenities,
            "available_rooms": h.available_rooms
        }
        for h in hotels
    ]
    
    return json.dumps({
        "status": "success",
        "city": city,
        "check_in": check_in,
        "check_out": check_out,
        "nights": nights,
        "guests": guests,
        "hotels_found": len(hotels_data),
        "hotels": hotels_data
    }, indent=2)


def get_weather_forecast(city: str, date: str) -> str:
    """
    Get weather forecast for a city on a specific date.
    
    Args:
        city: City name
        date: Date in YYYY-MM-DD format
    
    Returns:
        JSON string with weather forecast
    """
    print(f"☀️  Getting weather forecast for {city} on {date}")
    
    # Simulated weather data
    weather_data = {
        "Los Angeles": {
            "temperature": "75°F (24°C)",
            "condition": "Sunny",
            "precipitation": "0%",
            "humidity": "45%",
            "wind": "8 mph"
        },
        "Miami": {
            "temperature": "82°F (28°C)",
            "condition": "Partly Cloudy",
            "precipitation": "20%",
            "humidity": "70%",
            "wind": "12 mph"
        },
        "New York": {
            "temperature": "65°F (18°C)",
            "condition": "Clear",
            "precipitation": "5%",
            "humidity": "55%",
            "wind": "10 mph"
        }
    }
    
    # Find matching city
    forecast = None
    for city_name, data in weather_data.items():
        if city.lower() in city_name.lower() or city_name.lower() in city.lower():
            forecast = data
            break
    
    if not forecast:
        forecast = {
            "temperature": "72°F (22°C)",
            "condition": "Pleasant",
            "precipitation": "10%",
            "humidity": "50%",
            "wind": "5 mph"
        }
    
    return json.dumps({
        "city": city,
        "date": date,
        "forecast": forecast
    }, indent=2)


def calculate_trip_cost(
    flight_price: float,
    hotel_price: float,
    days: int,
    additional_expenses: float = 0.0
) -> str:
    """
    Calculate total trip cost including flights, hotel, and extras.
    
    Args:
        flight_price: Total flight cost (round-trip)
        hotel_price: Hotel price per night
        days: Number of days
        additional_expenses: Additional estimated expenses (default: 0.0)
    
    Returns:
        JSON string with cost breakdown
    """
    print(f"💰 Calculating trip cost for {days} days")
    
    hotel_total = hotel_price * days
    meals_estimate = 75 * days  # $75 per day for meals
    transport_estimate = 50 * days  # $50 per day for local transport
    
    subtotal = flight_price + hotel_total + additional_expenses
    estimated_daily = meals_estimate + transport_estimate
    total = subtotal + estimated_daily
    
    return json.dumps({
        "breakdown": {
            "flights": flight_price,
            "hotel": hotel_total,
            "meals_estimate": meals_estimate,
            "transport_estimate": transport_estimate,
            "additional": additional_expenses
        },
        "subtotal": subtotal,
        "estimated_daily_expenses": estimated_daily,
        "total_estimated_cost": total,
        "currency": "USD"
    }, indent=2)


def book_trip(
    flight_number: str,
    hotel_name: str,
    passenger_name: str,
    email: str,
    check_in: str,
    check_out: str
) -> str:
    """
    Book a complete trip (flight + hotel).
    
    Args:
        flight_number: Flight number to book
        hotel_name: Hotel name to book
        passenger_name: Passenger's full name
        email: Email for confirmation
        check_in: Check-in date
        check_out: Check-out date
    
    Returns:
        JSON string with booking confirmation
    """
    print(f"📝 Booking trip for {passenger_name}")
    
    booking_id = f"BOOK{datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    booking = {
        "booking_id": booking_id,
        "passenger_name": passenger_name,
        "email": email,
        "flight_number": flight_number,
        "hotel_name": hotel_name,
        "check_in": check_in,
        "check_out": check_out,
        "booking_date": datetime.now().isoformat(),
        "status": "confirmed"
    }
    
    BOOKINGS_DB[booking_id] = booking
    
    return json.dumps({
        "status": "success",
        "message": "Trip booked successfully!",
        "booking": booking
    }, indent=2)


def send_confirmation_email(
    email: str,
    booking_id: str,
    trip_details: str
) -> str:
    """
    Send booking confirmation email.
    
    Args:
        email: Recipient email
        booking_id: Booking reference ID
        trip_details: Summary of trip details
    
    Returns:
        JSON string with email status
    """
    print(f"📧 Sending confirmation email to {email}")
    
    return json.dumps({
        "status": "sent",
        "recipient": email,
        "booking_id": booking_id,
        "subject": f"Trip Confirmation - {booking_id}",
        "sent_at": datetime.now().isoformat(),
        "message": "Confirmation email sent successfully"
    }, indent=2)


def get_travel_tips(destination: str) -> str:
    """
    Get travel tips and recommendations for a destination.
    
    Args:
        destination: City or country name
    
    Returns:
        JSON string with travel tips
    """
    print(f"💡 Getting travel tips for {destination}")
    
    tips = {
        "Los Angeles": {
            "best_time_to_visit": "March to May, September to November",
            "must_visit": ["Hollywood Sign", "Santa Monica Pier", "Griffith Observatory"],
            "local_transport": "Rent a car recommended, Metro available",
            "safety_tips": "Generally safe, avoid downtown late at night",
            "local_cuisine": ["In-N-Out Burger", "Mexican food", "Food trucks"]
        },
        "Miami": {
            "best_time_to_visit": "December to May",
            "must_visit": ["South Beach", "Art Deco District", "Everglades"],
            "local_transport": "Uber/Lyft popular, trolley in beach areas",
            "safety_tips": "Safe in tourist areas, be aware of belongings on beach",
            "local_cuisine": ["Cuban food", "Seafood", "Key Lime Pie"]
        }
    }
    
    # Find matching destination
    destination_tips = None
    for city, data in tips.items():
        if destination.lower() in city.lower() or city.lower() in destination.lower():
            destination_tips = data
            break
    
    if not destination_tips:
        destination_tips = {
            "message": f"General tips for {destination}: Research local customs, check visa requirements, book accommodations in advance"
        }
    
    return json.dumps({
        "destination": destination,
        "tips": destination_tips
    }, indent=2)


print("✅ All travel booking functions defined!")
print("\n📋 Available functions:")
print("   • search_flights - Find available flights")
print("   • search_hotels - Search hotels")
print("   • get_weather_forecast - Check weather")
print("   • calculate_trip_cost - Calculate total cost")
print("   • book_trip - Make reservation")
print("   • send_confirmation_email - Send confirmation")
print("   • get_travel_tips - Get destination tips")


✅ All travel booking functions defined!

📋 Available functions:
   • search_flights - Find available flights
   • search_hotels - Search hotels
   • get_weather_forecast - Check weather
   • calculate_trip_cost - Calculate total cost
   • book_trip - Make reservation
   • send_confirmation_email - Send confirmation
   • get_travel_tips - Get destination tips


In [14]:
# AutoGen Travel Assistant with All Tools 

async def run_autogen_travel_assistant():
    """
    AutoGen implementation with comprehensive travel tools.
    """
    from autogen_agentchat.agents import AssistantAgent
    from autogen_core.tools import FunctionTool
    from autogen_ext.models.openai import OpenAIChatCompletionClient
    
    print("\n" + "="*70)
    print("🤖 AUTOGEN - SMART TRAVEL BOOKING ASSISTANT")
    print("="*70)
    
    # Create model client
    client = OpenAIChatCompletionClient(model="gpt-4o-mini")
    
    # Wrap all functions as tools
    tools = [
        FunctionTool(
            func=search_flights,
            description="Search for available flights between cities"
        ),
        FunctionTool(
            func=search_hotels,
            description="Search for hotels in a destination"
        ),
        FunctionTool(
            func=get_weather_forecast,
            description="Get weather forecast for a city"
        ),
        FunctionTool(
            func=calculate_trip_cost,
            description="Calculate total trip cost with breakdown"
        ),
        FunctionTool(
            func=book_trip,
            description="Book a complete trip (flight + hotel)"
        ),
        FunctionTool(
            func=send_confirmation_email,
            description="Send booking confirmation email"
        ),
        FunctionTool(
            func=get_travel_tips,
            description="Get travel tips and recommendations"
        )
    ]
    
    # Create travel assistant agent
    travel_agent = AssistantAgent(
        name="travel_assistant",
        model_client=client,
        tools=tools,
        system_message="""You are a professional travel booking assistant.

Your capabilities:
1. Search flights and hotels
2. Check weather forecasts
3. Calculate trip costs with detailed breakdowns
4. Book complete trips
5. Send confirmation emails
6. Provide travel tips and recommendations

Your workflow:
1. Understand the user's travel requirements
2. Search for flights and hotels
3. Check weather for the dates
4. Calculate total costs
5. Present options clearly with recommendations
6. Explain pricing and value

Be helpful, provide multiple options, and explain costs clearly.
Use tools to provide accurate, real-time information."""
    )
    
    # Example travel requests
    scenarios = [
        "I want to travel from New York to Los Angeles next month for 5 days. Can you help me find flights and hotels with costs?",
    ]
    
    print("\n🧳 Travel Request:")
    
    for i, scenario in enumerate(scenarios, 1):
        print(f"\n{'='*70}")
        print(f"Scenario {i}")
        print("="*70)
        print(f"\n👤 Customer: {scenario}\n")
        print("🤖 Travel Assistant:\n")
        print("-"*70)
        
        # Run agent WITHOUT termination_condition (not supported for single agent)
        result = await travel_agent.run(task=scenario)
        
        # Print all messages to see the conversation flow
        print("\n📜 Conversation Flow:")
        print("-"*70)
        for msg in result.messages:
            if hasattr(msg, 'content') and msg.content:
                role = getattr(msg, 'source', 'unknown')
                print(f"\n[{role}]: {msg.content[:200]}...")
        
        # Print the final response
        print("\n" + "="*70)
        print("✅ FINAL RESPONSE:")
        print("="*70)
        if result.messages:
            final_msg = result.messages[-1]
            if hasattr(final_msg, 'content'):
                print(final_msg.content)
            else:
                print(str(final_msg))
        
        print("\n" + "-"*70)
    
    print("\n✅ AutoGen demonstration completed!")
    print(f"📊 Total messages in conversation: {len(result.messages)}")
    
    return result


print("✅ AutoGen travel assistant defined (CORRECTED)!")
print("🚀 Run with: await run_autogen_travel_assistant()")


✅ AutoGen travel assistant defined (CORRECTED)!
🚀 Run with: await run_autogen_travel_assistant()


In [9]:
# Agent Framework Travel Assistant with @ai_function

async def run_agent_framework_travel_assistant():
    """
    Agent Framework implementation using @ai_function decorator.
    """
    from agent_framework import ai_function
    from agent_framework.openai import OpenAIChatClient
    
    print("\n" + "="*70)
    print("⚡ AGENT FRAMEWORK - SMART TRAVEL BOOKING ASSISTANT")
    print("="*70)
    
    # Decorate all functions with @ai_function
    # Note: We already defined them, so we'll reference them directly
    # In real code, you'd add @ai_function decorator to original definitions
    
    # For this demo, we'll create decorated versions
    @ai_function
    def search_flights_decorated(
        origin: str,
        destination: str,
        date: str,
        passengers: int = 1
    ) -> str:
        """Search for available flights between cities."""
        return search_flights(origin, destination, date, passengers)
    
    @ai_function
    def search_hotels_decorated(
        city: str,
        check_in: str,
        check_out: str,
        guests: int = 2
    ) -> str:
        """Search for hotels in a destination."""
        return search_hotels(city, check_in, check_out, guests)
    
    @ai_function
    def get_weather_decorated(city: str, date: str) -> str:
        """Get weather forecast for a city."""
        return get_weather_forecast(city, date)
    
    @ai_function
    def calculate_cost_decorated(
        flight_price: float,
        hotel_price: float,
        days: int,
        additional_expenses: float = 0.0
    ) -> str:
        """Calculate total trip cost."""
        return calculate_trip_cost(flight_price, hotel_price, days, additional_expenses)
    
    @ai_function
    def book_trip_decorated(
        flight_number: str,
        hotel_name: str,
        passenger_name: str,
        email: str,
        check_in: str,
        check_out: str
    ) -> str:
        """Book a complete trip."""
        return book_trip(flight_number, hotel_name, passenger_name, email, check_in, check_out)
    
    @ai_function
    def send_email_decorated(
        email: str,
        booking_id: str,
        trip_details: str
    ) -> str:
        """Send booking confirmation email."""
        return send_confirmation_email(email, booking_id, trip_details)
    
    @ai_function
    def get_tips_decorated(destination: str) -> str:
        """Get travel tips for a destination."""
        return get_travel_tips(destination)
    
    # Create client and agent
    client = OpenAIChatClient(model_id="gpt-4o-mini")
    
    travel_agent = client.create_agent(
        name="travel_assistant",
        instructions="""You are a professional travel booking assistant.

Your capabilities:
1. Search flights and hotels
2. Check weather forecasts
3. Calculate trip costs with detailed breakdowns
4. Book complete trips
5. Send confirmation emails
6. Provide travel tips and recommendations

Your workflow:
1. Understand the user's travel requirements
2. Search for flights and hotels
3. Check weather for the dates
4. Calculate total costs
5. Present options clearly
6. Book if user confirms
7. Send confirmation

Be helpful, provide options, and explain costs clearly.
Always show multiple options when available.""",
        tools=[
            search_flights_decorated,
            search_hotels_decorated,
            get_weather_decorated,
            calculate_cost_decorated,
            book_trip_decorated,
            send_email_decorated,
            get_tips_decorated
        ],
        model="gpt-4.1"
    )
    
    # Example travel request
    print("\n🧳 Travel Request:")
    request = "I want to travel from New York to Los Angeles next month for 5 days. Show me flight and hotel options with total costs."
    
    print(f"\n👤 Customer: {request}\n")
    print("⚡ Travel Assistant:\n")
    
    result = await travel_agent.run(request)
    print(result.text)
    
    print("\n" + "-"*70)
    print("✅ Agent Framework demonstration completed!")


print("✅ Agent Framework travel assistant defined!")
print("🚀 Run with: await run_agent_framework_travel_assistant()")


✅ Agent Framework travel assistant defined!
🚀 Run with: await run_agent_framework_travel_assistant()


In [15]:
# Complete end-to-end booking workflow demonstration

async def demo_complete_booking_workflow():
    """
    Demonstrate a complete booking workflow from search to confirmation.
    """
    from agent_framework import ai_function
    from agent_framework.openai import OpenAIChatClient
    
    print("\n" + "="*70)
    print("🎯 COMPLETE TRAVEL BOOKING WORKFLOW DEMONSTRATION")
    print("="*70)
    
    # Setup agent with decorated functions
    @ai_function
    def search_flights_ai(origin: str, destination: str, date: str, passengers: int = 1) -> str:
        """Search flights."""
        return search_flights(origin, destination, date, passengers)
    
    @ai_function
    def search_hotels_ai(city: str, check_in: str, check_out: str, guests: int = 2) -> str:
        """Search hotels."""
        return search_hotels(city, check_in, check_out, guests)
    
    @ai_function
    def get_weather_ai(city: str, date: str) -> str:
        """Get weather."""
        return get_weather_forecast(city, date)
    
    @ai_function
    def calculate_cost_ai(flight_price: float, hotel_price: float, days: int, additional: float = 0) -> str:
        """Calculate costs."""
        return calculate_trip_cost(flight_price, hotel_price, days, additional)
    
    @ai_function
    def book_trip_ai(flight_number: str, hotel_name: str, passenger_name: str, 
                     email: str, check_in: str, check_out: str) -> str:
        """Book trip."""
        return book_trip(flight_number, hotel_name, passenger_name, email, check_in, check_out)
    
    @ai_function
    def send_email_ai(email: str, booking_id: str, trip_details: str) -> str:
        """Send email."""
        return send_confirmation_email(email, booking_id, trip_details)
    
    @ai_function
    def get_tips_ai(destination: str) -> str:
        """Get tips."""
        return get_travel_tips(destination)
    
    client = OpenAIChatClient(model_id="gpt-4o-mini")
    agent = client.create_agent(
        name="booking_agent",
        instructions="""You are an expert travel booking agent. Follow this workflow:

1. SEARCH: Find flights and hotels based on user requirements
2. WEATHER: Check weather forecast for travel dates
3. CALCULATE: Provide detailed cost breakdown
4. RECOMMEND: Suggest the best options and explain why
5. BOOK: When user confirms, book the trip
6. CONFIRM: Send confirmation email
7. TIPS: Provide travel tips for the destination

Be conversational, helpful, and thorough.""",
        tools=[search_flights_ai, search_hotels_ai, get_weather_ai, 
               calculate_cost_ai, book_trip_ai, send_email_ai, get_tips_ai],
        model="gpt-4o-mini"
    )
    
    # Multi-turn conversation simulating complete booking
    conversation = [
        "I need to book a trip from New York to Los Angeles from December 25-30, 2025. I'm traveling with my wife (2 people). Can you help?",
        # Follow-up questions would go here in interactive mode
    ]
    
    print("\n📱 Starting Complete Booking Process\n")
    print("-"*70)
    
    for turn, message in enumerate(conversation, 1):
        print(f"\n👤 Customer (Turn {turn}): {message}\n")
        print("🤖 Travel Agent:\n")
        
        result = await agent.run(message)
        print(result.text)
        
        print("\n" + "-"*70)
    
    print("\n✅ Complete workflow demonstration finished!")
    print("\n💡 In a real application, this would continue with:")
    print("   • User selects preferred options")
    print("   • Agent books the selections")
    print("   • Confirmation email sent")
    print("   • Travel tips provided")


print("✅ Complete workflow demo defined!")
print("🚀 Run with: await demo_complete_booking_workflow()")


✅ Complete workflow demo defined!
🚀 Run with: await demo_complete_booking_workflow()


In [11]:
#Interactive travel assistant (chat interface)

async def interactive_travel_assistant():
    """
    Interactive travel assistant where you can chat naturally.
    """
    from agent_framework import ai_function
    from agent_framework.openai import OpenAIChatClient
    
    print("\n" + "="*70)
    print("💬 INTERACTIVE TRAVEL ASSISTANT")
    print("="*70)
    print("\nType your travel requests naturally. Examples:")
    print("  • Find me flights to Miami next week")
    print("  • What's the weather in Los Angeles?")
    print("  • Show me hotels in New York for 3 nights")
    print("  • Calculate cost for a 5-day trip to LA")
    print("\nType 'exit' to quit, 'bookings' to see all bookings\n")
    print("-"*70)
    
    # Setup decorated functions
    @ai_function
    def search_flights_ai(origin: str, destination: str, date: str, passengers: int = 1) -> str:
        """Search for flights."""
        return search_flights(origin, destination, date, passengers)
    
    @ai_function
    def search_hotels_ai(city: str, check_in: str, check_out: str, guests: int = 2) -> str:
        """Search for hotels."""
        return search_hotels(city, check_in, check_out, guests)
    
    @ai_function
    def get_weather_ai(city: str, date: str) -> str:
        """Get weather forecast."""
        return get_weather_forecast(city, date)
    
    @ai_function
    def calculate_cost_ai(flight_price: float, hotel_price: float, days: int, additional: float = 0) -> str:
        """Calculate trip costs."""
        return calculate_trip_cost(flight_price, hotel_price, days, additional)
    
    @ai_function
    def book_trip_ai(flight_number: str, hotel_name: str, passenger_name: str,
                     email: str, check_in: str, check_out: str) -> str:
        """Book a trip."""
        return book_trip(flight_number, hotel_name, passenger_name, email, check_in, check_out)
    
    @ai_function
    def send_email_ai(email: str, booking_id: str, trip_details: str) -> str:
        """Send confirmation email."""
        return send_confirmation_email(email, booking_id, trip_details)
    
    @ai_function
    def get_tips_ai(destination: str) -> str:
        """Get travel tips."""
        return get_travel_tips(destination)
    
    client = OpenAIChatClient(model_id="gpt-4o-mini")
    agent = client.create_agent(
        name="interactive_agent",
        instructions="""You are a friendly travel assistant. Help users:
- Search flights and hotels
- Check weather
- Calculate costs
- Book trips
- Get travel tips

Be conversational and helpful. Ask clarifying questions if needed.""",
        tools=[search_flights_ai, search_hotels_ai, get_weather_ai,
               calculate_cost_ai, book_trip_ai, send_email_ai, get_tips_ai],
        model="gpt-4o-mini"
    )
    
    # Interactive loop
    while True:
        try:
            user_input = input("\n👤 You: ").strip()
            
            if not user_input:
                continue
            
            if user_input.lower() in ['exit', 'quit', 'q']:
                print("\n✈️  Thank you for using our travel service! Safe travels!")
                break
            
            if user_input.lower() == 'bookings':
                print(f"\n📋 Total bookings: {len(BOOKINGS_DB)}")
                for booking_id, booking in BOOKINGS_DB.items():
                    print(f"\n  {booking_id}:")
                    print(f"    Passenger: {booking['passenger_name']}")
                    print(f"    Flight: {booking['flight_number']}")
                    print(f"    Hotel: {booking['hotel_name']}")
                    print(f"    Dates: {booking['check_in']} to {booking['check_out']}")
                continue
            
            print("\n🤖 Agent: ", end="", flush=True)
            result = await agent.run(user_input)
            print(result.text)
            
        except KeyboardInterrupt:
            print("\n\n✈️  Session interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")
            print("Please try again.")


print("✅ Interactive assistant defined!")
print("🚀 Run with: await interactive_travel_assistant()")


✅ Interactive assistant defined!
🚀 Run with: await interactive_travel_assistant()


In [16]:
# Show capabilities comparison

def show_capabilities_comparison():
    """Display what we've built and key features."""
    
    info = """
╔═══════════════════════════════════════════════════════════════════════════╗
║          SMART TRAVEL BOOKING ASSISTANT - FEATURES SHOWCASE               ║
╚═══════════════════════════════════════════════════════════════════════════╝

🎯 WHAT WE'VE BUILT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

A complete travel booking system with 7 interconnected tools:

1. 🔍 search_flights      → Find available flights
2. 🏨 search_hotels       → Search hotels
3. ☀️  get_weather        → Check weather forecast
4. 💰 calculate_trip_cost → Detailed cost breakdown
5. 📝 book_trip           → Make reservations
6. 📧 send_confirmation   → Email confirmations
7. 💡 get_travel_tips     → Destination recommendations


🚀 KEY CAPABILITIES DEMONSTRATED
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

✅ Function Calling (Tool Use)
   • AI agent automatically calls appropriate functions
   • Chains multiple function calls together
   • Passes data between functions

✅ Intelligent Tool Selection
   • Agent chooses which tools to use based on context
   • Can use multiple tools in sequence
   • Adapts to user requirements

✅ Data Flow Between Tools
   search_flights → calculate_cost → book_trip → send_confirmation
   (Price from search used in calculation, then booking, then email)

✅ Natural Language Understanding
   User: "I want to go to LA next month for 5 days"
   Agent: Extracts: destination, timeframe, duration
         Calls: search_flights, search_hotels, get_weather

✅ Complex Workflows
   1. Gather requirements
   2. Search options
   3. Calculate costs
   4. Present recommendations
   5. Confirm and book
   6. Send confirmation

✅ Error Handling
   • No flights found? Agent informs user
   • Missing information? Agent asks questions
   • Invalid dates? Agent clarifies


🎨 FRAMEWORK COMPARISON
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

AutoGen (Cell 6):
└─ Uses: FunctionTool wrapper
└─ Syntax: FunctionTool(func=search_flights, description="...")
└─ Best for: Complex multi-agent systems
└─ Overhead: Moderate (wrapper classes)

Agent Framework (Cells 7-9):
└─ Uses: @ai_function decorator
└─ Syntax: @ai_function + normal function
└─ Best for: Quick tool integration
└─ Overhead: Minimal (decorators only)


📊 REAL-WORLD APPLICATIONS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

This pattern works for:

Travel & Hospitality:
├─ Flight booking systems
├─ Hotel reservation platforms
├─ Tour package configurators
└─ Travel insurance systems

E-commerce:
├─ Product search + recommendations
├─ Inventory checking
├─ Order processing
└─ Shipping tracking

Finance:
├─ Account balance queries
├─ Transaction history
├─ Payment processing
└─ Investment recommendations

Customer Support:
├─ Order status lookup
├─ Return processing
├─ Appointment scheduling
└─ FAQ + knowledge base


💡 KEY LEARNINGS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. Function Calling = AI Agents with Superpowers
   Instead of just text, agents can now:
   • Query databases
   • Call APIs
   • Process transactions
   • Trigger actions

2. Tools Make Agents Practical
   Without tools: Just chat
   With tools: Actual business value

3. Natural Chaining
   Agent automatically chains tools:
   search → calculate → recommend → book → confirm

4. Type Safety Matters
   Strong typing = Better schema generation
   Good docstrings = Better tool selection

5. Both Frameworks Work Great
   Choose based on:
   • AutoGen: Multi-agent complexity
   • Agent Framework: Quick integration


🎯 TRY IT YOURSELF
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Cell 6:  await run_autogen_travel_assistant()
         → See AutoGen with FunctionTool

Cell 7:  await run_agent_framework_travel_assistant()
         → See Agent Framework with @ai_function

Cell 8:  await demo_complete_booking_workflow()
         → See complete end-to-end workflow

Cell 9:  await interactive_travel_assistant()
         → Chat naturally with the agent


🌟 EXTEND THIS PROJECT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Add more tools:
├─ @ai_function: cancel_booking()
├─ @ai_function: modify_booking()
├─ @ai_function: get_loyalty_points()
├─ @ai_function: apply_promo_code()
├─ @ai_function: request_special_assistance()
└─ @ai_function: add_travel_insurance()

Connect real APIs:
├─ Amadeus (flights)
├─ Booking.com (hotels)
├─ OpenWeatherMap (weather)
├─ SendGrid (email)
└─ Stripe (payments)


✨ PRODUCTION CONSIDERATIONS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

For production:
✓ Add authentication & authorization
✓ Implement rate limiting
✓ Add logging & monitoring
✓ Error handling & retries
✓ Database persistence (not in-memory)
✓ Payment gateway integration
✓ User session management
✓ Audit trails for bookings
✓ Compliance (PCI DSS, GDPR)
"""
    
    print(info)

show_capabilities_comparison()



╔═══════════════════════════════════════════════════════════════════════════╗
║          SMART TRAVEL BOOKING ASSISTANT - FEATURES SHOWCASE               ║
╚═══════════════════════════════════════════════════════════════════════════╝

🎯 WHAT WE'VE BUILT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

A complete travel booking system with 7 interconnected tools:

1. 🔍 search_flights      → Find available flights
2. 🏨 search_hotels       → Search hotels
3. ☀️  get_weather        → Check weather forecast
4. 💰 calculate_trip_cost → Detailed cost breakdown
5. 📝 book_trip           → Make reservations
6. 📧 send_confirmation   → Email confirmations
7. 💡 get_travel_tips     → Destination recommendations


🚀 KEY CAPABILITIES DEMONSTRATED
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

✅ Function Calling (Tool Use)
   • AI agent automatically calls appropriate functions
   • Chains multiple function calls together
   • Passes data betwee

In [ ]:
# Run AutoGen Travel Assistant
await run_autogen_travel_assistant()


🤖 AUTOGEN - SMART TRAVEL BOOKING ASSISTANT

🧳 Travel Request:

Scenario 1

👤 Customer: I want to travel from New York to Los Angeles next month for 5 days. Can you help me find flights and hotels with costs?

🤖 Travel Assistant:

----------------------------------------------------------------------

📜 Conversation Flow:
----------------------------------------------------------------------

[user]: I want to travel from New York to Los Angeles next month for 5 days. Can you help me find flights and hotels with costs?...

[travel_assistant]: Sure! Could you please provide me with the exact dates for your travel next month and how many passengers will be traveling? Additionally, do you have any specific preferences for hotels (e.g., budget...

✅ FINAL RESPONSE:
Sure! Could you please provide me with the exact dates for your travel next month and how many passengers will be traveling? Additionally, do you have any specific preferences for hotels (e.g., budget, star rating)?

-----------

TaskResult(messages=[TextMessage(id='c9e126a0-2585-470f-b6e7-fdc06e276e70', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 12, 18, 13, 46, 9, 688945, tzinfo=datetime.timezone.utc), content='I want to travel from New York to Los Angeles next month for 5 days. Can you help me find flights and hotels with costs?', type='TextMessage'), TextMessage(id='78f5b0a9-939a-4382-ba03-adfc9a9b00cf', source='travel_assistant', models_usage=RequestUsage(prompt_tokens=588, completion_tokens=44), metadata={}, created_at=datetime.datetime(2025, 12, 18, 13, 46, 12, 191219, tzinfo=datetime.timezone.utc), content='Sure! Could you please provide me with the exact dates for your travel next month and how many passengers will be traveling? Additionally, do you have any specific preferences for hotels (e.g., budget, star rating)?', type='TextMessage')], stop_reason=None)

In [18]:
# Run Agent Framework Travel Assistant
await run_agent_framework_travel_assistant()


⚡ AGENT FRAMEWORK - SMART TRAVEL BOOKING ASSISTANT

🧳 Travel Request:

👤 Customer: I want to travel from New York to Los Angeles next month for 5 days. Show me flight and hotel options with total costs.

⚡ Travel Assistant:

✈️  Searching flights: New York → Los Angeles on 2024-11-01
🏨 Searching hotels in Los Angeles (2024-11-01 to 2024-11-06)
It seems that there were no flights available from New York to Los Angeles for your selected date (November 1, 2024). However, I found some hotel options for your stay in Los Angeles from November 1 to November 6 for 5 nights. Here are the details:

### Hotel Options in Los Angeles:

1. **Hollywood Luxury Hotel**
   - **Location:** Hollywood, Los Angeles
   - **Rating:** 4.5
   - **Price per Night:** $250.00
   - **Total Price for 5 Nights:** $1,250.00
   - **Amenities:** Pool, Gym, Free WiFi, Restaurant
   - **Available Rooms:** 10

2. **Downtown Budget Inn**
   - **Location:** Downtown, Los Angeles
   - **Rating:** 3.8
   - **Price per Night:*

In [19]:
# Run Complete Booking Workflow
await demo_complete_booking_workflow()


🎯 COMPLETE TRAVEL BOOKING WORKFLOW DEMONSTRATION

📱 Starting Complete Booking Process

----------------------------------------------------------------------

👤 Customer (Turn 1): I need to book a trip from New York to Los Angeles from December 25-30, 2025. I'm traveling with my wife (2 people). Can you help?

🤖 Travel Agent:

✈️  Searching flights: New York → Los Angeles on 2025-12-25
🏨 Searching hotels in Los Angeles (2025-12-25 to 2025-12-30)
It seems that I wasn't able to find any flights from New York to Los Angeles for your selected dates (December 25-30, 2025). However, I found some great hotel options for you in Los Angeles:

### Hotels Available

1. **Hollywood Luxury Hotel**
   - **Location:** Hollywood, Los Angeles
   - **Rating:** 4.5/5
   - **Price per Night:** $250
   - **Total Price for 5 Nights:** $1,250
   - **Amenities:** Pool, Gym, Free WiFi, Restaurant
   - **Available Rooms:** 10

2. **Downtown Budget Inn**
   - **Location:** Downtown, Los Angeles
   - **Rating:**

In [20]:
# Launch Interactive Chat Interface
await interactive_travel_assistant()


💬 INTERACTIVE TRAVEL ASSISTANT

Type your travel requests naturally. Examples:
  • Find me flights to Miami next week
  • What's the weather in Los Angeles?
  • Show me hotels in New York for 3 nights
  • Calculate cost for a 5-day trip to LA

Type 'exit' to quit, 'bookings' to see all bookings

----------------------------------------------------------------------

🤖 Agent: I'd be happy to help you with your booking! Could you please provide me with the following details?

1. **Flight Information:**
   - Origin (Departure city)
   - Destination (Arrival city)
   - Date of travel
   - Number of passengers

2. **Hotel Information:**
   - City for hotel booking
   - Check-in date
   - Check-out date
   - Number of guests

3. **Passenger Information:**
   - Name of the passenger
   - Email address for confirmation

Once I have these details, I'll get started on your booking!

🤖 Agent: It seems you want to cancel something, but I'll need a bit more information. Are you looking to cancel a

A production-ready travel booking system showcasing:

- ✅ 7 interconnected tools working together
- ✅ Intelligent tool chaining (search → calculate → book → confirm)
- ✅ Natural language interface (no forms needed!)
- ✅ Both frameworks (AutoGen + Agent Framework)
- ✅ Complete workflows (end-to-end booking)
- ✅ Interactive chat (real conversations)

This demonstrates the true power of function calling - transforming AI from just chat to actionable business systems!